In [ ]:
import sys

sys.path.append("..")
from core.DataLoader import (
    DataPreprocessor,
    DataConfig,
    LoadConfig,
    get_load_config_from_yaml,
)
import numpy as np
from importlib import reload
import matplotlib.pyplot as plt
import yaml
import core.assignment_models as Models
import core
import keras
import tensorflow as tf
import os


PLOTS_DIR = f"plots/toponium_transformer_HLF/"
MODEL_DIR = f"models/toponium_transformer_HLF/"
CONFIG_PATH = "../config/workspace_config_HLF.yaml"

if not os.path.exists(PLOTS_DIR):
    os.makedirs(PLOTS_DIR)
if not os.path.exists(MODEL_DIR):
    os.makedirs(MODEL_DIR)


load_config = get_load_config_from_yaml(CONFIG_PATH)

DataProcessor = DataPreprocessor(load_config)


plt.rcParams.update({"font.size": 14})

data_config = DataProcessor.load_from_npz(
    load_config["toponium"], max_events=1_000_000
)
X_train, y_train, X_val, y_val = DataProcessor.split_data(test_size=0.1)
del DataProcessor

2025-11-28 15:36:45.782737: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764340605.805464 3003856 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764340605.812787 3003856 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1764340605.831861 3003856 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764340605.831888 3003856 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1764340605.831891 3003856 computation_placer.cc:177] computation placer alr

In [2]:
reload(Models)
reload(core)
TransformerMatcher = Models.FeatureConcatTransformer(data_config, name="ToponiumTransformer")

TransformerMatcher.build_model(
    hidden_dim=64,
    num_layers=6,
    num_heads=8,
    dropout_rate=0.1
)

TransformerMatcher.adapt_normalization_layers(X_train)
#TransformerMatcher.load_model(MODEL_DIR + "model.keras")

TransformerMatcher.compile_model(
    loss={
        "assignment": core.utils.AssignmentLoss(lambda_excl=0.0),
    },
    optimizer=keras.optimizers.AdamW(learning_rate=1e-4, weight_decay=1e-4),
    metrics={
        "assignment": [core.utils.AssignmentAccuracy()],
    },
)

FeatureConcatTransformer is designed for classification tasks; regression targets will be ignored.


I0000 00:00:1764340629.516705 3003856 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15511 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:65:00.0, compute capability: 6.0


Building model without regression output.


In [3]:
TransformerMatcher.train_model(
    epochs=100,
    X_train=X_train,
    y_train=y_train,
    sample_weights=core.utils.compute_sample_weights(X_train, y_train),
    batch_size=1024,
)

Epoch 1/100


I0000 00:00:1764340648.862250 3004435 service.cc:152] XLA service 0x7f37a00124e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764340648.862291 3004435 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
2025-11-28 15:37:29.454350: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-11-28 15:37:30.138735: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
I0000 00:00:1764340651.919296 3004435 cuda_dnn.cc:529] Loaded cuDNN version 91500


  8/702 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.1432 - loss: 0.1972 

I0000 00:00:1764340665.726238 3004435 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


701/702 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.2677 - loss: 0.1575

2025-11-28 15:37:57.614641: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


702/702 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.2678 - loss: 0.1575

2025-11-28 15:38:14.703684: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert
2025-11-28 15:38:16.712271: W tensorflow/compiler/tf2xla/kernels/assert_op.cc:39] Ignoring Assert operator FeatureConcatTransformerModel_1/met_input_transform_1/assert_equal_1/Assert/Assert


702/702 ━━━━━━━━━━━━━━━━━━━━ 64s 46ms/step - accuracy: 0.2680 - loss: 0.1575 - val_accuracy: 0.5767 - val_loss: 0.1081
Epoch 2/100
702/702 ━━━━━━━━━━━━━━━━━━━━ 12s 17ms/step - accuracy: 0.5285 - loss: 0.1173 - val_accuracy: 0.6527 - val_loss: 0.0953
Epoch 3/100
702/702 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.6112 - loss: 0.1048 - val_accuracy: 0.6876 - val_loss: 0.0904
Epoch 4/100
702/702 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6434 - loss: 0.0991 - val_accuracy: 0.7040 - val_loss: 0.0877
Epoch 5/100
702/702 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6582 - loss: 0.0957 - val_accuracy: 0.7101 - val_loss: 0.0851
Epoch 6/100
702/702 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6671 - loss: 0.0935 - val_accuracy: 0.7127 - val_loss: 0.0837
Epoch 7/100
702/702 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6765 - loss: 0.0915 - val_accuracy: 0.7233 - val_loss: 0.0821
Epoch 8/100
702/702 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.6826 - loss: 0.0899 - val_

In [4]:
TransformerMatcher.save_model(MODEL_DIR + "model.keras")

Model saved to models/toponium_transformer_HLF/model.keras
